# Microscope Acquisition with Pycro-manager

This notebook demonstrates how to control the microscope using the high-level `Acquisition` API.
## Requirements
1. Micro-Manager running with Pycro-manager server enabled.
2. `pycromanager` installed.
3. Testing hardware: XY stage `MS2000`, Camera `Dhyana 401D`. Hardware drivers installed.

In [1]:
import numpy as np
from pycromanager import Core, Acquisition
import random
import time

In [ ]:
# Connect to the Micro-Manager Core to get initial state
core = Core()
print(core)

xy_stage = core.get_xy_stage_device()
print(f"Using XY Stage: {xy_stage}")

if xy_stage:
    start_x = core.get_x_position(xy_stage)
    start_y = core.get_y_position(xy_stage)
    print(f"Initial Position: X={start_x}, Y={start_y}")
else:
    # Fallback for testing if no stage present
    start_x, start_y = 0, 0
    print("No XY Stage found. Using (0,0).")

Using XY Stage: XYStage
Initial Position: X=-1885.7, Y=4329.3


In [ ]:
import random
import math

def generate_acquisition_events(mode='random', num_rounds=5, moves_per_round=10, step_size=10, start_x=0, start_y=0):
    """
    Generate acquisition events based on the specified mode.
    """
    events = []
    static_factor = 0.01
    for r in range(num_rounds):
        curr_x, curr_y = start_x, start_y
        theta = random.uniform(0, 2 * math.pi)
        
        for m in range(moves_per_round):
            if mode == 'random':
                dx = random.uniform(-step_size, step_size)
                dy = random.uniform(-step_size, step_size)
                curr_x += dx
                curr_y += dy
                target_x = curr_x
                target_y = curr_y
                type_str = 'random_relative'

            elif mode == 'static':
                dx = random.uniform(-step_size*static_factor, step_size*static_factor)
                dy = random.uniform(-step_size*static_factor, step_size*static_factor)
                target_x = start_x + dx
                target_y = start_y + dy
                type_str = 'static'

            elif mode == 'linear':
                dist = (m + 1) * step_size
                target_x = start_x + dist * math.cos(theta)
                target_y = start_y + dist * math.sin(theta)
                type_str = 'linear_move'
            else:
                raise ValueError(f"Unknown mode: {mode}")

            evt = {
                'axes': {'round': r, 'position': m, 'type': type_str},
                'x': target_x, # positions in um
                'y': target_y, # positions in um
                'exposure': 10,
            }
            
            # Tag the last event of the round so the hook knows to return home
            if m == moves_per_round - 1:
                evt['return_after'] = True
            
            events.append(evt)
        
        # NOTE: Explicit 'return_home' event removed to skip acquisition.
        # The return move is handled by the acquisition hook.
        
    return events


In [ ]:
# Define modes and parameters
mode = 'random'  # Options: 'random', 'relative', 'linear'
num_rounds = 5
moves_per_round = 10
step_size = 10

print(f"Generating events for mode: {mode}")
events_list = generate_acquisition_events(
    mode=mode, 
    num_rounds=num_rounds, 
    moves_per_round=moves_per_round, 
    step_size=step_size,
    start_x=start_x,
    start_y=start_y
)
print(f"Generated {len(events_list)} events.")
# Show first few events
print(events_list[:3])

In [4]:
# Define a directory to save data (optional)
save_dir = r"C:\Users\pchen406\Codes\WF-Acq-py\AcquisitionData"
acq_name = "RandomWalkAcquisition"

def return_to_start_hook(event, bridge, event_queue):
    """
    Hook to move stage back to start after the last event of a round.
    Run post-camera, so image is taken at the random pos, then we return.
    """
    if 'return_after' in event and event['return_after']:
        core = bridge.get_core()
        print(f"[Hook] Round complete. Returning to start: ({start_x:.1f}, {start_y:.1f})")
        try:
            # Use the global variables for device name and start pos
            # Note: xy_stage must be defined (it is from cell 2)
            core.set_xy_position(xy_stage, start_x, start_y)
            core.wait_for_device(xy_stage)
        except Exception as e:
            print(f"[Hook] Error returning to start: {e}")
    
    return event

# Run the Acquisition
print("Starting Acquisition...")
with Acquisition(directory=save_dir, name=acq_name, post_camera_hook_fn=return_to_start_hook) as acq:
    acq.acquire(events_list)
    
print("Acquisition Complete.")